# Estimating Graphical Models using Score Matching

In [1]:
import HD, NPGM

## Experiments with Gaussian graphical model

The following function generates n samples from a p-dimensional Normal distribution with covariance structure $\sigma_{ab}=\rho^{|a-b|}$

In [2]:
function gen_data_chain(n::Int64, p::Int64, rho::Float64; zero_thr=1e-4)

    Sigma = zeros(Float64, p, p)
    for a=1:p
        for b=1:p
            Sigma[a,b] = rho^abs(a-b)
        end
    end
    Theta = inv(Sigma)
    Theta[find(abs(Theta) .< zero_thr)] = 0.
    
    X = randn(n, p) * sqrtm(Sigma), Sigma, Theta
end;

In [23]:
function getGaussianParameters(beta0, beta1, p)
    
  assert(length(beta0) == 2*p)
  assert(length(beta1) == int(p*(p-1)/2))
         
  mu = zeros(p)
  precM = zeros(p, p)
  for a = 1:p
    mu[a] = beta0[2*a]
    precM[a, a] = beta0[2*a-1]
  end    
    
  indEdge = 0
  for a = 1:p
    for b = a+1:p
      indEdge += 1
      t = beta1[indEdge]
      precM[a,b] = t
      precM[b,a] = t
    end
  end
    
  mu, precM
end

getGaussianParameters (generic function with 1 method)

In [48]:
K = 2
L = 1
n = 500
p = 80
rho = 0.8
numColumns = int(p*K+p*(p-1)/2*L)

3320

The function below solves 
$$ \min \frac{1}{2} \theta^T A \theta - b^T \theta + \lambda \|\theta_2\|_1$$
where $\theta = (\theta_1^T, \theta_2^T)^T$. The first part of $\theta$, $\theta_1 \in \mathrm{R}^p*K$ where $K$ is the number of basis functions for the node. For example, in the setting where we are estimating the Gaussian graphical model, one has $K=2$, since we have $x_a^2$ and $x_a$ as functions corresponding to a node.

In [43]:
function step_one_inference(A, b, p, K, L; lambda=0.3)    
    # project first p*K coordinates 
    A00 = cholfact( A[1:p*K, 1:p*K] )    
    Ap = A[p*K+1:end, p*K+1:end] - A[p*K+1:end, 1:p*K]*(A00 \ A[1:p*K, p*K+1:end])
    Ap = full(Ap)
    bp = b[p*K+1:end] - A[p*K+1:end, 1:p*K] * (A00 \ b[1:p*K])
    
    numcolumns = int(p*(p-1)/2*L)
    beta1 = spzeros(numcolumns, 1)   
    lambdaArr = fill(lambda, numcolumns)
    HD.lasso!(beta1, Ap, bp, lambdaArr)         
    
    beta0 = - (A00 \ (A[1:p*K, p*K+1:end] * beta1 - b[1:p*K]))
    
    getGaussianParameters(beta0, beta1, p)
end

step_one_inference (generic function with 1 method)

In [54]:
numRep = 10
lambdaArr = logspace(log10(3), log10(0.3), 30)
numLambda = length(lambdaArr)
pathSol = cell(numRep, numLambda)

@time X, tCov, tPrec = gen_data_chain(n,p,rho)
@time DD = NPGM.getDD(X, K, L, NPGM.gm_node_der_f, NPGM.gm_edge_der_f)
@time E = NPGM.getE(X, K, L, NPGM.gm_node_der_2_f, NPGM.gm_edge_der_2_f);

# scale E by -1 since it corresponds to Xy in Lasso code
scale!(E, -1./n)
scale!(DD, 1./n);

elapsed time: 0.04227355 seconds (1286208 bytes allocated)
elapsed time: 1.098883981 seconds (228587072 bytes allocated, 11.09% gc time)
elapsed time: 0.349683184 seconds (255138688 bytes allocated, 39.61% gc time)


In [55]:
@time emu, eprecM = step_one_inference(DD, E, p, K, L; lambda=0.45)

elapsed time: 0.325647337 seconds (212249320 bytes allocated, 12.49% gc time)


([-0.0971084,0.121478,0.018503,-0.00173752,0.0232006,0.0346955,0.0117008,-0.0478683,0.0446604,0.0277244  …  0.0409763,0.00883592,-0.00155439,0.198094,-0.0611422,0.0298203,0.0394289,-0.064821,0.128905,-0.0222653],
80x80 Array{Float64,2}:
  2.13281   -1.3622     -0.139798   …   0.0       0.0       0.0    
 -1.3622     3.30789    -1.52133        0.0       0.0       0.0    
 -0.139798  -1.52133     3.5554         0.0       0.0       0.0    
  0.0       -0.0303765  -1.57618        0.0       0.0       0.0    
  0.0        0.0        -0.0169526      0.0       0.0       0.0    
  0.0        0.0         0.0        …   0.0       0.0       0.0    
  0.0        0.0         0.0            0.0       0.0       0.0    
  0.0        0.0         0.0            0.0       0.0       0.0    
  0.0        0.0         0.0            0.0       0.0       0.0    
  0.0        0.0         0.0            0.0       0.0       0.0    
  0.0        0.0         0.0        …   0.0       0.0       0.0    
  0.0        0.

In [28]:
tPrec

3x3 Array{Float64,2}:
  2.77778  -2.22222   0.0    
 -2.22222   4.55556  -2.22222
  0.0      -2.22222   2.77778

In [57]:
countnz(eprecM)

272

In [14]:
size(DD)

(495,495)